Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к нам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Необходимо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.
Часть открытых данных есть, например, на сайте archive.org:
https://archive.org/details/201309_foursquare_dataset_umn
Будем пользоваться этими данными:

In [4]:
import pandas as pd
from scipy.spatial.distance import euclidean
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

In [5]:
columns = ['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at']

In [6]:
data = pd.read_csv('checkins.dat', sep='|', skiprows=[0, 1],
                   header=None, parse_dates=['created_at'],
                   names=columns, skipinitialspace=True, low_memory=False)
data.dropna(axis=0, how='any', inplace=True)
data = data.astype({'id':int, 'user_id':int, 'venue_id':int})

In [7]:
print(data.info())
data.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 1 to 1021964
Data columns (total 6 columns):
id            396634 non-null int64
user_id       396634 non-null int64
venue_id      396634 non-null int64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 21.2 MB
None


,id,user_id,venue_id,latitude,longitude,created_at
1021959,955561,626076,20073,40.850100,-73.866246,2012-04-13 09:56:48
1021960,955892,674797,2297,33.748995,-84.387982,2012-04-13 10:56:03
1021961,956377,845102,11195,42.765366,-71.467566,2012-04-13 12:08:45
1021962,956119,1139114,29488,42.439479,-83.743830,2012-04-13 11:36:44
1021964,956733,960666,60,42.331427,-83.045754,2012-04-13 21:56:19


Formed subsample of 100000 rows

In [8]:
s_data = data[['id','latitude','longitude']][:100000]
del data

In [9]:
s_data.set_index('id', inplace=True)

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [10]:
est = MeanShift(bandwidth=0.1)

In [11]:
est.fit(s_data)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [12]:
res = pd.DataFrame(est.labels_)

In [13]:
r = np.unique(est.labels_, return_counts=True)
clusters = [est.cluster_centers_[x] for x,y in zip(r[0],r[1]) if y>15]

20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [14]:
offices = np.array([[33.751277, -118.188740], [25.867736, -80.324116], [51.503016, -0.075479], [52.378894, 4.885084], [39.366487, 117.036146], [-33.868457, 151.205134]])

In [15]:
offices

array([[ 3.37512770e+01, -1.18188740e+02],
       [ 2.58677360e+01, -8.03241160e+01],
       [ 5.15030160e+01, -7.54790000e-02],
       [ 5.23788940e+01,  4.88508400e+00],
       [ 3.93664870e+01,  1.17036146e+02],
       [-3.38684570e+01,  1.51205134e+02]])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [16]:
rslt_frame = pd.DataFrame.from_records(
[[(min([euclidean(o, clu) for clu in clusters])),
(np.argmin(np.array([euclidean(o, clu) for clu in clusters]))),
(clusters[np.argmin(np.array([euclidean(o, clu) for clu in clusters]))])] for o in offices],
    columns=['distance', 'cluster_id', 'coordinate'])

In [17]:
rslt_frame

,distance,cluster_id,coordinate
0,0.070848,51,"[33.8098779552631, -118.14892380690813]"
1,0.022674,412,"[25.8456722642857, -80.31889059642857]"
2,0.050058,58,"[51.502991260887086, -0.12553728870967767]"
3,0.009353,372,"[52.37296399032261, 4.892317222580647]"
4,9.267575,429,"[31.230393000000017, 121.473704]"
5,0.007835,403,"[-33.86063042857143, 151.20477592857145]"


Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [18]:
rslt_frame[rslt_frame.distance == rslt_frame.distance.min()].coordinate

5    [-33.86063042857143, 151.20477592857145]
Name: coordinate, dtype: object

In [19]:
%%writefile 'answ_banners.txt'
-33.868457 151.205134

Overwriting 'answ_banners.txt'
